# **elice 군 장병 AI SW 온라인 교육 - 프로젝트 사례대회 참여**

> 작성자 이름 : 김지욱
>
> 작성자 소속 : 일병
>
> 작성자 계급 : 미8군사령부 본부대대 작전중대

## **분석계획안**

### **선행연구**

> * 서울시 그늘막 설치 적합지역 분석, 이제승, 홍익대학교, 2017 (https://bigdata.seoul.go.kr/noti/selectNoti.do?r_id=P260&bbs_seq=274&ac_type=A2&sch_type=&sch_text=&currentPage=1)
>
> * 폭염으로 인한 기후변화 취약계층의 사망률 변화 분석, 고려대학교, 2013 (https://bigdata.seoul.go.kr/noti/selectNoti.do?r_id=P260&bbs_seq=274&ac_type=A2&sch_type=&sch_text=&currentPage=1)
>
> * 폭염과 서울시민의 생활양식 변화, 서울연구원, 2020 (https://bigdata.seoul.go.kr/noti/selectNoti.do?r_id=P260&bbs_seq=274&ac_type=A2&sch_type=&sch_text=&currentPage=1)

</br>

### **가용데이터**

**1\) 성북구청 제공 데이터(정보공개법에 따른 정보공개청구제도 활용)**

> * 서울특별시 성북구 그늘막쉼터 설치 현황
> * 서울특별시 성북구 폭염저감시설(그늘막) 설치 및 운영 계획
> * 서울특별시 성북구 그늘막 설치·관리 지침 개정

**2\) 디지털정보처 제공 데이터**

> * 개설과목 제공 데이터
> * 랜드마크와 라운지 데이터

**3\) 공공데이터포털 제공 데이터**

> * 기상청_생활기상지수 조회서비스(3.0) API (https://www.data.go.kr/data/15085288/openapi.do)
> * 서울특별시_서울시 대로변 횡단보도 위치정보(https://www.data.go.kr/data/15098157/openapi.do)
> * 서울시 도로노선 정보(http://data.seoul.go.kr/dataList/OA-15496/S/1/datasetView.do#AXexec)
> * 서울시 보도 정보(서울시에서 DB 개편으로 22년 말까지 제공 중단)


**4\) 서울열린데이터광장 제공 데이터**

> * 집계구 단위 서울 생활인구(내국인) (https://data.seoul.go.kr/dataList/OA-14979/F/1/datasetView.do)

**5\) 통계지리정보서비스 제공 데이터**

> * 집계구경계 SHP 데이터 (https://sgis.kostat.go.kr/view/pss/openDataIntrcn)

</br>

### **구현목표**
> * 예산기반 모델
>
> * 확률기반 모델


<img src='https://drive.google.com/uc?export=download&id=1iQlQxZZr9QPDRjbYDf7WvjmY5F9F1ldM' width="" height ="" />

In [9]:
import json
#
import pandas as pd
pd.set_option('mode.chained_assignment',  None)
#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#
import folium






"""

0. 데이터 불러오기

"""

# 데이터 불러오기
df_data = pd.read_excel('./data_processed/무제 폴더/df_final.xlsx', index_col='Unnamed: 0')


# feature와 label을 지정함.
column_feature = ['total_population', 'elder_composition', 'fever_index']
column_label = ['Installation']


# 
df_data_feature = df_data[column_feature]
df_data_label = df_data[column_label]






"""

1. 머신러닝 모델링

"""

# np.array로 변환함.
df_data_feature_numpy = df_data_feature.to_numpy()
df_data_label_numpy = df_data_label.to_numpy()


# train_set, test_set 분리함.
train_feature, test_feature, train_label, test_label = train_test_split(df_data_feature_numpy, df_data_label_numpy, random_state=42)


# 정규화함.
ss = StandardScaler()
train_feature_scaled = ss.fit_transform(train_feature)
test_feature_scaled = ss.fit_transform(test_feature)


# 로지스틱 회귀모형을 모델링함.
lr = LogisticRegression()
lr.fit(train_feature_scaled, train_label.ravel())






"""

2. 모델링 결과값 추가 및 내림차순 정렬

"""

# input 데이터 전처리
df_input = df_data_feature
df_input_numpy = df_input.to_numpy()
df_input_scaled = ss.fit_transform(df_input_numpy)


# 확률값 예측 및 데이터프레임 만들기
np_predict_proba = lr.predict_proba(df_input_scaled)
df_predict_proba = pd.DataFrame(np_predict_proba)


df_combined = pd.concat([df_data, df_predict_proba], axis=1)


# 설치수량 및 확률 처리하기
container_max_prob = []
container_install_num = []
for i,e in enumerate(df_combined.iterrows()):
    installation = e[1]['Installation']
    start_index = 9 + installation + 1
    slicing_range = e[1][start_index:]
    try:
        max_prob = max(e[1][start_index:])
        install_num = list(slicing_range).index(max_prob) + installation + 1
    except:
        max_prob = 0
    container_max_prob.append(max_prob)
    container_install_num.append(install_num)
    
    
# 데이터프레임에 추가
df_predict_proba['max_prob'] = container_max_prob
df_predict_proba['install_num'] = container_install_num






"""

3. 예산에 기반한, 신규설치수량 산출 알고리즘 구현

"""

# 단위 행정기관인 '구'의 개념을, 경영회계학 도메인 지식을 활용하여, 클래스와 매소드로 구현
class Gu:

        # total_num은 신규 설치 이전의 기존 수량을 입력해 주세요.
    def __init__(self, total, additional, re_install, maintenance, install_cost, public_operation_cost, maintenance_cost):
        self.total = total
        self.additional = additional
        self.re_install = re_install
        self.maintenance = maintenance
        self.each_installation_cost = (install_cost + public_operation_cost) / (additional + re_install)
        self.each_maintenance_cost = maintenance_cost / maintenance

        # 차기 연도의 신규설치수량을 산출하는 메소드
    def show_me_next_year_numbers(self, total_cost, reinstall_num, maintenance_num):
        maintenance_cost_ny = (maintenance_num * self.each_maintenance_cost)
        installation_cost_ny = total_cost - maintenance_cost_ny

        ## 단순히 '구매'할 수 있는 숫자
        number = int(installation_cost_ny / self.each_installation_cost) - reinstall_num
        return number

    
    
    
    
    
"""

4. 지도정보를 시각화하는 클래스와 메소드를 구현

"""

class Map:
    
    def __init__(self, num_new, df_data, df_predict_proba):
        self.num_new = num_new
        self.df_data = df_data
        self.df_predict_proba = df_predict_proba
    
    
    #
    def get_df_output(self):
    
        #
        df_temp = pd.concat([self.df_data, self.df_predict_proba], axis=1)
        df_temp_extracted = df_temp[['ADM_CD', 'TOT_REG_CD', 'Installation', 'install_num', 'max_prob']]
        df_output_target_column_astype = df_temp_extracted.astype({'TOT_REG_CD':'string'})
        
        #
        df_temp_extracted_sorted = df_temp_extracted.sort_values('max_prob', ascending=False)
        df_focus = df_temp_extracted_sorted.iloc[:self.num_new, :]

        
        #
        target_local_code = list(df_focus['TOT_REG_CD'])
        container_focus = []
        for i,e in enumerate(df_temp_extracted.iterrows()):
            local_code = e[1][1]
            if local_code in target_local_code:
                container_focus.append(1)
            else:
                container_focus.append(0)
        df_temp_extracted.loc[:,'target_local_code'] = container_focus

    
        #
        df_output_selected = df_temp_extracted[df_temp_extracted['target_local_code']==1]
        df_output_selected_astype = df_output_selected.astype({'TOT_REG_CD':'string'})

        
        #
        return df_output_target_column_astype, df_output_selected_astype
    
    
    
    # 지도정보를 시각화하는 함수를 구현
    def get_map_by_budget(self):
        
        #
        df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
        df_output_target_column_astype, df_output_selected_astype = self.get_df_output()
    
        #
        state_geo = './data_processed/무제 폴더/map_as_json.geojson'
        start = (37.606768, 127.024438)
    
        #
        m = folium.Map(location=start, tiles='cartodbpositron', zoom_start=13)
        folium.TileLayer('Stamen Toner').add_to(m)
        fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)
        # folium.TileLayer('Stamen Terrain').add_to(m)
        # folium.TileLayer('Stamen Toner').add_to(m)
        # # folium.TileLayer('Stamen Water Color').add_to(m)
        # folium.TileLayer('cartodbpositron').add_to(m)
        # folium.TileLayer('cartodbdark_matter').add_to(m)
    
    
        #
        with open(state_geo,mode='rt',encoding='utf-8') as f:
            geo = json.loads(f.read())
            f.close()
            
        #
        choropleth_recommendation = folium.Choropleth(
            line_weight=0.3,
            line_opacity=0.2,
            line_color='gray',
            nan_fill_opacity=0,
            name='설치장소로 제안된 집계구 영역',
            bins=[0, 1/3, 2/3, 1],
            geo_data=state_geo, 
            data = df_output_selected_astype, 
            columns=('TOT_REG_CD', 'target_local_code'),
            fill_color='Blues',
            key_on='feature.properties.TOT_REG_CD',
            legend_name='설치대상으로 제안된 집계구 영역(boolean)',
            fill_opacity = 0.9,
            smooth_factor=0
            ).add_to(m)
        
        #
        fg2 = folium.FeatureGroup(name='설치된 위치 마커',overlay=True, show=False).add_to(m)
        for i in df_installation.index:
            folium.Marker(
                location = list(df_installation.loc[i,['lat','lng']]),
                tooltip = list(df_installation.loc[i,['Location']])[0],
                icon=folium.Icon(color='gray', icon='ok', icon_color='white')
            ).add_to(fg2) 
        
        #
        df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
        fg3 = folium.FeatureGroup(name='설치가능한 위치 마커', overlay=True, show=False).add_to(m)
        for i in df_crossroad.index:
            folium.Marker(
                location = list(df_crossroad.loc[i,['y','x']]),
                tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
                icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
            ).add_to(fg3)
        
        #
        choropleth_outline = folium.Choropleth(
            line_weight=0.3,
            line_opacity=0.2,
            nan_fill_opacity=0,
            name='집계구 영역',
            geo_data=state_geo, 
            columns=('TOT_REG_CD', 'target_local_code'),
            key_on='feature.properties.TOT_REG_CD',
            fill_opacity = 0.05,
            smooth_factor=0
            ).add_to(m)
    
        #
        style_function = "font-size: 12px; font-weight: bold"
        choropleth_recommendation.geojson.add_child(
            folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
        choropleth_outline.geojson.add_child(
            folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))

        #
        folium.LayerControl().add_to(m)

        #
        display(m)


    # 지도정보를 시각화하는 함수를 구현
    def get_map_by_proba(self):
        
        #
        df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
        df_output_target_column_astype, df_output_selected_astype = self.get_df_output()
    
        #
        state_geo = './data_processed/무제 폴더/map_as_json.geojson'
        start = (37.606768, 127.024438)
    
        #
        m = folium.Map(location=start, tiles='cartodbpositron', zoom_start=13)
        folium.TileLayer('Stamen Toner').add_to(m)
        fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)
        # folium.TileLayer('cartodbpositron').add_to(m)
        # folium.TileLayer('Stamen Terrain').add_to(m)
        # folium.TileLayer('Stamen Toner').add_to(m)
        # folium.TileLayer('Stamen Water Color').add_to(m)
        # folium.TileLayer('cartodbdark_matter').add_to(m)
    
    
        #
        with open(state_geo,mode='rt',encoding='utf-8') as f:
            geo = json.loads(f.read())
            f.close()
        
        #
        legend = df_predict_proba.describe()['max_prob']
        # proba_cut = legend['50%']
        # df_predict_proba_cut = df_predict_proba.loc[df_predict_proba['max_prob'] >= proba_cut]
        # legend_cut = df_predict_proba_cut.describe()['max_prob']
        legend_range = [legend['min'], legend['25%'], legend['50%'], legend['75%'], legend['max']]
        
        #
        choropleth_recommendation = folium.Choropleth(
            line_weight=0.5,
            line_opacity=0.5,
            nan_fill_opacity=0,
            name='확률로 산출된 집계구 영역',
            bins=legend_range,
            geo_data=state_geo, 
            data = df_output_target_column_astype, 
            columns=('TOT_REG_CD', 'max_prob'),
            fill_color='RdYlGn',
            key_on='feature.properties.TOT_REG_CD',
            legend_name='집계구 영역에 관한 그늘막쉼터 설치 확률(float)',
            fill_opacity = 0.4,
            smooth_factor=0
            ).add_to(m)
        
        #
        df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
        fg3 = folium.FeatureGroup(name='설치가능한 위치 마커', overlay=True, show=False).add_to(m)
        for i in df_crossroad.index:
            folium.Marker(
                location = list(df_crossroad.loc[i,['y','x']]),
                tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
                icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
            ).add_to(fg3)
        

        #
        fg2 = folium.FeatureGroup(name='설치된 위치 마커', overlay=True, show=False).add_to(m)
        for i in df_installation.index:
            folium.Marker(
                location = list(df_installation.loc[i,['lat','lng']]),
                tooltip = list(df_installation.loc[i,['Location']])[0],
                icon=folium.Icon(color='gray', icon='ok', icon_color='white')
            ).add_to(fg2)        

            
        #
        choropleth_outline = folium.Choropleth(
            line_weight=0.3,
            line_opacity=0.2,
            nan_fill_opacity=0,
            name='집계구 영역',
            geo_data=state_geo, 
            columns=('TOT_REG_CD', 'target_local_code'),
            key_on='feature.properties.TOT_REG_CD',
            fill_opacity = 0.05,
            smooth_factor=0
            ).add_to(m)
        
        #
        style_function = "font-size: 12px; font-weight: bold"
        choropleth_recommendation.geojson.add_child(
            folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
        choropleth_outline.geojson.add_child(
            folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))

        #
        folium.LayerControl().add_to(m)

        #
        display(m)
    


#
def get_now_map():
      
    #
    state_geo = './data_processed/무제 폴더/map_as_json.geojson'
    start = (37.606768, 127.024438)
    
    #
    # df_installation = pd.read_excel('./설치장소.xlsx')
    # df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
        
    #
    m = folium.Map(location=start, tiles=None, zoom_start=13)
    fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)
    # fg2 = folium.FeatureGroup(name='설치된 위치 마커',overlay=True).add_to(m)
    folium.TileLayer('cartodbpositron').add_to(m)
    folium.TileLayer('Stamen Toner').add_to(m)
    # folium.TileLayer('Stamen Terrain').add_to(m)
    # folium.TileLayer('Stamen Water Color').add_to(m)
    # folium.TileLayer('cartodbdark_matter').add_to(m)
            
    #
    df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
    fg2 = folium.FeatureGroup(name='설치된 위치 마커',overlay=True).add_to(m)
    for i in df_installation.index:
        folium.Marker(
            location = list(df_installation.loc[i,['lat','lng']]),
            tooltip = list(df_installation.loc[i,['Location']])[0],
            icon=folium.Icon(color='gray', icon='ok', icon_color='white')
        ).add_to(fg2)
    
    #
    df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
    fg3 = folium.FeatureGroup(name='설치가능한 위치 마커',overlay=True, show=False).add_to(m)
    for i in df_crossroad.index:
        folium.Marker(
            location = list(df_crossroad.loc[i,['y','x']]),
            tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
            icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
        ).add_to(fg3)
    
    #
    choropleth_outline = folium.Choropleth(
        line_weight=0.3,
        line_opacity=0.5,
        nan_fill_opacity=0,
        name='집계구 영역',
        geo_data=state_geo, 
        columns=('TOT_REG_CD', 'target_local_code'),
        fill_color='Greens',
        key_on='feature.properties.TOT_REG_CD',
        fill_opacity = 0.30,
        smooth_factor=0
        ).add_to(m)
    
    #
    style_function = "font-size: 12px; font-weight: bold"
    choropleth_outline.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
    
    #
    folium.LayerControl().add_to(m)
    
    #
    display(m)


def get_map_for_total_population():

    state_geo = './data_processed/무제 폴더/map_as_json.geojson'
    start = (37.606768, 127.024438)

    df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
    df_data = pd.read_excel('./data_processed/무제 폴더/df_final.xlsx')
    df_data = df_data.astype({'TOT_REG_CD':'string', 'total_population':'int'})

    m = folium.Map(location=start, tiles='cartodbpositron', zoom_start=13)
    folium.TileLayer('Stamen Toner').add_to(m)
    fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)


    with open(state_geo,mode='rt',encoding='utf-8') as f:
        geo = json.loads(f.read())
        f.close()


    # legend = df_data.describe()['total_population']
    # legend_range = [legend['min'], legend['25%'], legend['50%'], legend['75%'], legend['max']]
    
    
    choropleth_population = folium.Choropleth(
        line_weight=0.5,
        # bins=legend_range,
        line_opacity=0.5,
        nan_fill_opacity=0,
        name='생활인구(전체연령)',
        geo_data=state_geo, 
        data = df_data, 
        columns=('TOT_REG_CD', 'total_population'),
        fill_color='Blues',
        key_on='feature.properties.TOT_REG_CD',
        legend_name='생활인구(전체연령)(int)',
        fill_opacity = 0.4,
        smooth_factor=0
        ).add_to(m)

        #
    df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
    fg3 = folium.FeatureGroup(name='설치가능한 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_crossroad.index:
        folium.Marker(
            location = list(df_crossroad.loc[i,['y','x']]),
            tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
            icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
        ).add_to(fg3)
        

        #
    fg2 = folium.FeatureGroup(name='설치된 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_installation.index:
        folium.Marker(
            location = list(df_installation.loc[i,['lat','lng']]),
            tooltip = list(df_installation.loc[i,['Location']])[0],
            icon=folium.Icon(color='gray', icon='ok', icon_color='white')
        ).add_to(fg2)        

            
        #
    choropleth_outline = folium.Choropleth(
        line_weight=0.3,
        line_opacity=0.2,
        nan_fill_opacity=0,
        name='집계구 영역',
        geo_data=state_geo, 
        columns=('TOT_REG_CD', 'target_local_code'),
        key_on='feature.properties.TOT_REG_CD',
        fill_opacity = 0.05,
        smooth_factor=0
        ).add_to(m)
        
        #
    style_function = "font-size: 12px; font-weight: bold"
    choropleth_population.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
    choropleth_outline.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))

    folium.LayerControl().add_to(m)
    
    display(m)

    
    
def get_map_for_elder_population():

    state_geo = './data_processed/무제 폴더/map_as_json.geojson'
    start = (37.606768, 127.024438)

    df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
    df_data = pd.read_excel('./data_processed/무제 폴더/df_final.xlsx')
    df_data = df_data.astype({'TOT_REG_CD':'string', 'elder_composition':'float'})

    m = folium.Map(location=start, tiles='cartodbpositron', zoom_start=13)
    folium.TileLayer('Stamen Toner').add_to(m)
    fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)


    with open(state_geo,mode='rt',encoding='utf-8') as f:
        geo = json.loads(f.read())
        f.close()


    # legend = df_data.describe()['total_population']
    # legend_range = [legend['min'], legend['25%'], legend['50%'], legend['75%'], legend['max']]
    
    
    choropleth_population = folium.Choropleth(
        line_weight=0.5,
        # bins=legend_range,
        line_opacity=0.5,
        nan_fill_opacity=0,
        name='생활인구 비중(무더위 취약계층)',
        geo_data=state_geo, 
        data = df_data, 
        columns=('TOT_REG_CD', 'elder_composition'),
        fill_color='Purples',
        key_on='feature.properties.TOT_REG_CD',
        legend_name='생활인구 비중(무더위 취약계층)(float)',
        fill_opacity = 0.4,
        smooth_factor=0
        ).add_to(m)

        #
    df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
    fg3 = folium.FeatureGroup(name='설치가능한 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_crossroad.index:
        folium.Marker(
            location = list(df_crossroad.loc[i,['y','x']]),
            tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
            icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
        ).add_to(fg3)
        

        #
    fg2 = folium.FeatureGroup(name='설치된 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_installation.index:
        folium.Marker(
            location = list(df_installation.loc[i,['lat','lng']]),
            tooltip = list(df_installation.loc[i,['Location']])[0],
            icon=folium.Icon(color='gray', icon='ok', icon_color='white')
        ).add_to(fg2)        

            
        #
    choropleth_outline = folium.Choropleth(
        line_weight=0.3,
        line_opacity=0.2,
        nan_fill_opacity=0,
        name='집계구 영역',
        geo_data=state_geo, 
        columns=('TOT_REG_CD', 'target_local_code'),
        key_on='feature.properties.TOT_REG_CD',
        fill_opacity = 0.05,
        smooth_factor=0
        ).add_to(m)
        
        #
    style_function = "font-size: 12px; font-weight: bold"
    choropleth_population.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
    choropleth_outline.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))

    folium.LayerControl().add_to(m)
    
    display(m)
    

     
def get_map_for_fever_index():

    state_geo = './data_processed/무제 폴더/map_as_json.geojson'
    start = (37.606768, 127.024438)

    df_installation = pd.read_excel('./data_processed/무제 폴더/설치장소.xlsx')
    df_data = pd.read_excel('./data_processed/무제 폴더/df_final.xlsx')
    df_data = df_data.astype({'TOT_REG_CD':'string', 'fever_index':'float'})

    m = folium.Map(location=start, tiles='cartodbpositron', zoom_start=13)
    folium.TileLayer('Stamen Toner').add_to(m)
    fg1 = folium.FeatureGroup(name='remove map',overlay=False).add_to(m)


    with open(state_geo,mode='rt',encoding='utf-8') as f:
        geo = json.loads(f.read())
        f.close()


    # legend = df_data.describe()['total_population']
    # legend_range = [legend['min'], legend['25%'], legend['50%'], legend['75%'], legend['max']]
    
    
    choropleth_population = folium.Choropleth(
        line_weight=0.5,
        # bins=10,
        line_opacity=0.5,
        nan_fill_opacity=0,
        name='열 지수(float)',
        geo_data=state_geo, 
        data = df_data, 
        columns=('TOT_REG_CD', 'fever_index'),
        fill_color='OrRd',
        key_on='feature.properties.TOT_REG_CD',
        legend_name='열 지수(float)',
        fill_opacity = 0.4,
        smooth_factor=0
        ).add_to(m)

        #
    df_crossroad = pd.read_csv('./data_processed/무제 폴더/서울시 대로변 횡단보도 위치정보(point).csv', encoding='utf-8')
    fg3 = folium.FeatureGroup(name='설치가능한 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_crossroad.index:
        folium.Marker(
            location = list(df_crossroad.loc[i,['y','x']]),
            tooltip = list(df_crossroad.loc[i,['노드 ID']])[0],
            icon=folium.Icon(color='lightblue', icon='search', icon_color='white')
        ).add_to(fg3)
        

        #
    fg2 = folium.FeatureGroup(name='설치된 위치 마커', overlay=True, show=False).add_to(m)
    for i in df_installation.index:
        folium.Marker(
            location = list(df_installation.loc[i,['lat','lng']]),
            tooltip = list(df_installation.loc[i,['Location']])[0],
            icon=folium.Icon(color='gray', icon='ok', icon_color='white')
        ).add_to(fg2)        

            
        #
    choropleth_outline = folium.Choropleth(
        line_weight=0.3,
        line_opacity=0.2,
        nan_fill_opacity=0,
        name='집계구 영역',
        geo_data=state_geo, 
        columns=('TOT_REG_CD', 'target_local_code'),
        key_on='feature.properties.TOT_REG_CD',
        fill_opacity = 0.05,
        smooth_factor=0
        ).add_to(m)
        
        #
    style_function = "font-size: 12px; font-weight: bold"
    choropleth_population.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))
    choropleth_outline.geojson.add_child(
        folium.features.GeoJsonTooltip(['label'], style=style_function, labels=False))

    folium.LayerControl().add_to(m)
    
    display(m)



"""

5. UI 구현

"""
import ipywidgets as widgets
from ipywidgets import *
# from ipywidgets import interact
import pandas as pd
import plotly.express as px



tab = widgets.Tab()
tab_name = ('표지', '0. 활용안내', '1. UI구현 및 실행코드', '2. 가용데이터 및 분석')
for i,e in enumerate(tab_name):
    tab.set_title(i, e)


#
html_title = widgets.HTML(value=f"<h1>교외를 위한 그늘막쉼터의 최적설치장소 제안</h1>")
html_br = widgets.HTML(value="</br>")
html_logo = widgets.HTML(value=f"""
<div align="right">
    <img src='https://drive.google.com/uc?export=download&id=1PWyYWWESkAWbTBd8PR_t6-2KMxLsDS-B' width="10%" height ="" />
</div>
""")
    
    
# UI 컴포넌트 - tab_box1
html_cover_title = widgets.HTML(value=f"""
<div align="center">
  <h1>2022 KU 스마트캠퍼스 데이터톤</h1>
  <h1>분석보고서 및 개념증명서</h1>
</div>
""")
html_cover_content = widgets.HTML(value=f"""
<div align="center">
    <h3>9조 bada</h3>
    <h3>"고려대학교 학우와 안암동 주민을 위한, 그늘막쉼터의 입지 선정 제안"</h3>
    <p>경영학과 2017120029 김지욱</p>
    <p>경영학과 2017120066 김종현</p>
    <p>경영학과 2019120010 이지원</p>
    <p>경영학과 2010120134 조수현</p>
    </br>
    </br>
    </br>
    </br>
    <h3>제1부. <교외를 위한 그늘막쉼터의 최적설치장소 제안> 편</h3>
    </br>
    </br>
    </br>
    </br>
</div>
""")

#
tab_box1 = widgets.VBox(
    [
        html_br,
        html_cover_title,
        html_br,
        html_cover_content,
        html_br,
        html_logo,
    ]
)




# UI 컴포넌트 - tab_box2
html_header = widgets.HTML(value=f"<h2>{tab_name[1]}</h2>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>본 단계에서는, <b>"교외를 위한 그늘막쉼터의 최적설치장소"</b>가 표현된 <b>UI를 활용하기 위한 가이드라인</b>을 안내합니다.</li>
    <li>본 단계에서는, <b>"교외를 위한 그늘막쉼터의 최적설치장소"</b>를 제안하기 위한 <b>가용데이터와 그 분석내용</b>을 안내합니다.</li>
  </ol>
</div>
""")

#
html_subtitle_1 = widgets.HTML(value=f"<h2>1. UI를 활용하기 위한 가이드라인</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <ol>
    <li>UI를 활용하기 위해서는, 데이터를 입력하여 <b>행정구 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 생성된 행정구 인스턴스 객체에서 <b>신규설치수량을 산출하는 메소드</b>를 호출해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 시각화에 필요한 <b>지도정보체계 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 적절한 정보유형을 선택하여 <b>위치정보시각화 분석결과</b>를 출력할 수 있습니다.</li>
  </ol>
</div>
""")
#
html_subtitle_2 = widgets.HTML(value=f"<h2>2. 가용데이터와 그 분석내용</h2>")
html_subcontent_2 = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>기상지수 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>인구통계 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>사회적가치 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>법제도 기준</b>와 관련된 토글 <b>옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")

#
tab_box2 = widgets.VBox(
    [        
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        html_br,
        
        html_subtitle_2,
        html_subcontent_2,
        html_br,
        html_logo
    ]
)




# UI 컴포넌트 - tab_box3
html_header = widgets.HTML(value=f"<h2>{tab_name[1]}</h2>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>UI를 활용하기 위해서는, 데이터를 입력하여 <b>행정구 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 생성된 행정구 인스턴스 객체에서 <b>신규설치수량을 산출하는 메소드</b>를 호출해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 시각화에 필요한 <b>지도정보체계 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 적절한 정보유형을 선택하여 <b>위치정보시각화 분석결과</b>를 출력할 수 있습니다.</li>
  </ol>
</div>
""")


#
html_subtitle_1 = widgets.HTML(value="<h2>1. 행정구 인스턴스 객체 생성하기.</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <p>본 단계에서는 직전 회계연도에서의 데이터를 'Gu' 클래스에 담아 인스턴스 객체를 생성합니다.</p>
</div>
""")
#
int_text_previous_total = widgets.IntText(description='전체 수량 : ', value=113)
int_text_previous_additional = widgets.IntText(description='신규 수량 : ', value=5)
int_text_previous_re_install = widgets.IntText(description='재설치 수량 : ', value=7)
int_text_previous_maintenance = widgets.IntText(description='점검 수량 : ', value=9)
int_text_previous_install_cost = widgets.IntText(description='설치 비용 : ', value=22800000)
int_text_previous_public_operation_cost = widgets.IntText(description='설비 비용 : ', value=5000000)
int_text_previous_maintenance_cost = widgets.IntText(description='유지 비용 : ', value=9000000)
#
button_instance = widgets.Button(description='회계연도 객체 생성하기')
out_instance = widgets.Output()
def on_button_instance_clicked(_):
    global gu_target
    with out_instance:
        out_instance.clear_output()
        gu_target = Gu(int_text_previous_total.value, int_text_previous_additional.value, int_text_previous_re_install.value, int_text_previous_maintenance.value, int_text_previous_install_cost.value, int_text_previous_public_operation_cost.value, int_text_previous_maintenance_cost.value)
        print(f'직전 회계연도 객체 생성 완료 : {gu_target}')
button_instance.on_click(on_button_instance_clicked)


#
html_subtitle_2 = widgets.HTML(value="<h2>2. 신규설치수량을 산출하는 메소드 호출하기.</h2>")
html_subcontent_2 = widgets.HTML(value="""
<div>
  <p>본 단계에서는 생성한 인스턴스 객체에서 메소드를 호출하여 신규설치가 가능한 그늘막쉼터의 숫자를 리턴합니다.</p>
</div>
""")
#
int_text_budget = widgets.IntText(description='현재 예산 : ', value=40000000)
int_text_reinstall = widgets.IntText(description='재설치 수량 : ', value=5)
int_text_maintanence = widgets.IntText(description='점검 수량 : ', value=5)
#
button_calculation = widgets.Button(description='신규설치 수량 계산하기')
out_calculation = widgets.Output()
def on_button_calculation_clicked(_):
    global num_new
    with out_calculation:
        out_calculation.clear_output()
        num_new = gu_target.show_me_next_year_numbers(int_text_budget.value, int_text_reinstall.value, int_text_maintanence.value)
        print(f'신규설치 가능수량 : {num_new}개')
button_calculation.on_click(on_button_calculation_clicked)


#
html_subtitle_3 = widgets.HTML(value="<h2>3. 지도정보체계 인스턴스 객체 생성하기.</h2>")
html_subcontent_3 = widgets.HTML(value="""
<div>
  <p>본 단계에서는 모델링에서의 결과를 'Map' 클래스에 담아 인스턴스 객체를 생성합니다.</p>
</div>
""")
#
button_draw_map = widgets.Button(description='지도정보 객체 생성하기')
out_draw_map = widgets.Output()
def on_button_draw_map_clicked(_):
    global map_recommendation
    with out_draw_map:
        out_draw_map.clear_output()
        map_recommendation = Map(num_new, df_data, df_predict_proba)
        print(f'지도정보 객체 생성 완료 : {map_recommendation}')
button_draw_map.on_click(on_button_draw_map_clicked)

#
html_subtitle_4 = widgets.HTML(value="<h2>4. 위치정보시각화 분석결과 출력하기.</h2>")
html_subcontent_4 = widgets.HTML(value="""
<div>
  <p>본 단계에서는 모델링에서의 결과를 'Map' 클래스에 담아 인스턴스 객체를 생성합니다.</p>
</div>
<div>
    <h4>팁</h4>
    <ul>
        <li>지도의 우측 상단의 옵션을 통해, 지도유형을 변경할 수 있습니다.</li>
        <li>지도의 우측 상단의 옵션을 통해, 정보유형을 표현하거나 숨길 수 있습니다.</li>
    </ul>
    <h4>주의사항</h4>
    <ul>
        <li>만약 지도정보 인스턴스 객체가 생성되지 않은 경우, 지도정보체계가 출력되지 않는 오류가 발생할 수도 있습니다.</li>
        <li>만약 지도가 정상적으로 표시되지 않는 경우, 다른 유형의 지도를 선택한 이후에 다시 해당 유형의 지도를 누르시면 지도가 표시될 수도 있습니다.</li>
        <li>만약 본 문서에 기술된 주의사항 이외의 문제가 발생하는 경우, 실행환경의 kernel을 초기화 한 이후에 다시 실행할 것을 권장합니다.</li>
    </ul>
</div>
""")
#
toggle_option = ['그늘막쉼터 설치 현황', '예산기반 최적위치 보기', '확률기반 집계구 보기']
toggle_button = widgets.ToggleButtons(options=toggle_option, value=None)
#
def set_state(options):
    global map_recommendation
    if options == '그늘막쉼터 설치 현황':
        get_now_map()
    elif options == '예산기반 최적위치 보기':
        try:
            map_recommendation.get_map_by_budget()
        except:
            print('Error - Map 인스턴스가 생성되지 않았습니다.')
    elif options == '확률기반 집계구 보기':
        try:
            map_recommendation.get_map_by_proba()
        except:
            print('Error - Map 인스턴스가 생성되지 않았습니다.')
    elif options == None:
        result = None

f_subcontent_4 = interactive(set_state, options=toggle_button);

# UI 컴포넌트 - tab
tab_box3 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        int_text_previous_total,
        int_text_previous_additional,
        int_text_previous_re_install,
        int_text_previous_maintenance,
        int_text_previous_install_cost,
        int_text_previous_public_operation_cost,
        int_text_previous_maintenance_cost,
        button_instance,
        out_instance,
        html_br,
        
        
        html_subtitle_2,
        html_subcontent_2,
        int_text_budget,
        int_text_reinstall,
        int_text_maintanence,
        button_calculation,
        out_calculation,
        html_br,
        
        html_subtitle_3,
        html_subcontent_3,
        button_draw_map,
        out_draw_map,
        html_br,
        
        html_subtitle_4,
        html_subcontent_4,
        f_subcontent_4,
        html_br,
        html_logo
    ]
)





# UI 컴포넌트 - tab_box4
# html_header = widgets.HTML(value=f"<h1>{tab_name[2]}</h1>")
html_header = widgets.HTML(value=f"<h2>{tab_name[1]}</h2>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>기상지수 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>인구통계 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>사회적가치 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>법제도 기준</b>와 관련된 토글 <b>옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")


#
html_subtitle_1 = widgets.HTML(value=f"<h2>1. 기상지수 기준</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <ul>
    <li><b>기상청_생활기상지수 조회서비스(3.0) API</b>에서 제공하는 <b>자외선 지수 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ul>
</div>
""")

#
button_draw_map_1 = widgets.Button(description='지도 표시하기')
out_draw_map_1 = widgets.Output()
def on_button_draw_map_clicked_1(_):
    # global map_recommendation
    with out_draw_map_1:
        out_draw_map_1.clear_output()
        print(f'기상청_생활기상지수 조회서비스(3.0) API - 자외선 지수 데이터')
        # get_map_for_elder_population()
button_draw_map_1.on_click(on_button_draw_map_clicked_1)


#
html_subtitle_2 = widgets.HTML(value=f"<h2>2. 인구통계 기준</h2>")
html_subcontent_2 = widgets.HTML(value="""
<div>
  <ul>
    <li><b>집계구 단위 서울 생활인구(내국인)</b>에서 제공하는 <b>생활인구 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ul>
</div>
""")

#
toggle_option = ['4', '5', '6']
toggle_button = widgets.ToggleButtons(options=toggle_option, tooltips=toggle_option, value=None)
#
# def set_state(option):
#     if option == '4':
#         get_map_for_population()
#     elif option == '5':
#         get_map_for_rest_area()
#     elif option == '6':
#         get_map_for_ratio_of_population_to_rest_area()
#     else:
#         return None
# f_subcontent_2 = interactive(set_state, option=toggle_button);
button_draw_map_2 = widgets.Button(description='지도 표시하기')
out_draw_map_2 = widgets.Output()
def on_button_draw_map_clicked_2(_):
    # global map_recommendation
    with out_draw_map_2:
        out_draw_map_2.clear_output()
        # map_recommendation = Map(num_new, df_data, df_predict_proba)
        print(f'집계구 단위 서울 생활인구(내국인) - 생활인구 데이터')
        get_map_for_total_population()
button_draw_map_2.on_click(on_button_draw_map_clicked_2)


#
html_subtitle_3 = widgets.HTML(value=f"<h2>3. 법제도 기준</h2>")
html_subcontent_3 = widgets.HTML(value="""
<div>
  <ul>
    <li><b>서울시 도로노선 정보</b>에서 제공하는 <b>도로폭 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li><b>서울시 보도 정보</b>에서 제공하는 <b>보도폭 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ul>
  <p>서울시에서 DB서버 개편작업으로 인해 데이터제공이 중단되었습니다.ㅠㅜ</p>
</div>
""")
button_draw_map_3 = widgets.Button(description='지도 표시하기', disabled=True)


#
html_subtitle_4 = widgets.HTML(value=f"<h2>4. 사회적가치 기준</h2>")
html_subcontent_4 = widgets.HTML(value="""
<div>
  <ul>
    <li><b>집계구 단위 서울 생활인구(내국인)</b>에서 제공하는 <b>생활인구(만65세 이상) 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li><b>기상청_생활기상지수 조회서비스(3.0) API</b>에서 제공하는 <b>열 지수 데이터</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ul>
</div>
""")

button_draw_map_4 = widgets.Button(description='지도 표시하기')
out_draw_map_4 = widgets.Output()
def on_button_draw_map_clicked_4(_):
    # global map_recommendation
    with out_draw_map_4:
        out_draw_map_4.clear_output()
        print(f'집계구 단위 서울 생활인구(내국인) - 생활인구(만65세 이상) 데이터')
        get_map_for_elder_population()
button_draw_map_4.on_click(on_button_draw_map_clicked_4)

button_draw_map_5 = widgets.Button(description='지도 표시하기')
out_draw_map_5 = widgets.Output()
def on_button_draw_map_clicked_5(_):
    # global map_recommendation
    with out_draw_map_5:
        out_draw_map_5.clear_output()
        print(f'기상청_생활기상지수 조회서비스(3.0) API - 열 지수 데이터')
        get_map_for_fever_index()
button_draw_map_5.on_click(on_button_draw_map_clicked_5)




#
tab_box4 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        button_draw_map_1,
        out_draw_map_1,
        html_br,
        
        html_subtitle_2,
        html_subcontent_2,
        button_draw_map_2,
        out_draw_map_2,
        html_br,
        
        html_subtitle_3,
        html_subcontent_3,
        button_draw_map_3,
        html_br,
        
        html_subtitle_4,
        html_subcontent_4,
        button_draw_map_4,
        out_draw_map_4,
        button_draw_map_5,
        out_draw_map_5,
        html_br,
        
        
        html_logo,
    ]
)


# # UI 컴포넌트 - tab_box5
# html_header = widgets.HTML(value=f"<h2>{tab_name[4][3:]}</h2>")
# html_cover_title = widgets.HTML(value=f"""
# <div align="center">
#   <h1>2022 KU 스마트캠퍼스 데이터톤</h1>
#   <h1>분석보고서 및 개념증명서</h1>
# </div>
# """)
# html_content = widgets.HTML(value=f"""
# <div align="center">
#     <h3>9조 bada</h3>
#     <h3>"고려대학교 학우와 안암동 주민을 위한, 그늘막쉼터의 입지 선정 제안"</h3>
#     <p>경영학과 2017120029 김지욱</p>
#     <p>경영학과 2017120066 김종현</p>
#     <p>경영학과 2019120010 이지원</p>
#     <p>경영학과 2010120134 조수현</p>
#     </br>
#     </br>
#     </br>
#     <h3>제1부. <교외를 위한 그늘막쉼터의 최적설치장소 제안> 편</h3>
#     </br>
#     </br>
# </div>
# """)

#
tab_box5 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_logo,
    ]
)


children = [tab_box1, tab_box2, tab_box3, tab_box4]
tab.children = children
display(tab)

In [14]:
"""

5. UI 구현

"""
import ipywidgets as widgets
from ipywidgets import *
import pandas as pd
import plotly.express as px


"""

교내인구의 시계열 분석

'./data_processed/df_lecture_population.xlsx'

"""
def get_map_for_population():

    #
    df_data = pd.read_excel('./data_processed/무제 폴더/df_lecture_population.xlsx')

    #
    fig = px.scatter_mapbox(
        df_data, 
        lat='위도', 
        lon='경도', 
        color='수강인원',
        size='수강인원',
        center={'lat':37.587636, 
                'lon':127.029174},
        zoom=15,
        color_continuous_scale=px.colors.sequential.Blues, 
        animation_frame='강의시간',
        hover_name='강의건물',
        # width=600,
        height=850,
        mapbox_style="carto-positron"
    )

    #
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig.show()


"""

교내휴게공간의 위치정보시각화 분석

'./data_processed/df_lounge_number.xlsx'

"""
def get_map_for_rest_area():
    df_data = pd.read_excel('./data_processed/무제 폴더/df_lounge_number.xlsx')
    df_data = df_data.drop('Unnamed: 0', axis=1)

    fig = px.scatter_mapbox(
        df_data, 
        lat='위도', 
        lon='경도', 
        color='의자 수',
        size='의자 수',
        center={'lat':37.587636, 
                'lon':127.029174},
        zoom=15,
        color_continuous_scale=px.colors.sequential.Blugrn, 
        hover_name='교내장소',
        # width=600,
        height=850,
        mapbox_style="carto-positron",
        labels='aa',
    )


    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig.show()


    
"""

교내휴게공간 대비 교내인구 비율의 시계열-위치정보시각화 분석

'./data_processed/df_rest_area.xlsx'

"""
def get_map_for_ratio_of_population_to_rest_area():

    #
    df_data = pd.read_excel('./data_processed/무제 폴더/df_rest_area.xlsx')
    df_data = df_data.drop('Unnamed: 0', axis=1)

    #
    fig = px.scatter_mapbox(
        df_data, 
        lat='위도', 
        lon='경도', 
        color='혼잡도',
        size='혼잡도',
        center={'lat':37.587636, 
                'lon':127.029174},
        zoom=15,
        color_continuous_scale=px.colors.sequential.Reds, 
        hover_name='강의건물',
        # width=600,
        height=850,
        mapbox_style="carto-positron",
        animation_frame='강의시간',
    )
    
    #
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig.show()
    






# def set_state(x):
#     print(x)

tab = widgets.Tab()
tab_name = ('표지', '0. 활용안내', '1. UI구현 및 실행코드', '2. 가용데이터 및 분석', '3. 결론 및 요약')
for i,e in enumerate(tab_name):
    tab.set_title(i, e)


#
html_title = widgets.HTML(value=f"<h1>교내를 위한 그늘막벤치의 최적설치장소 제안</h1>")
html_br = widgets.HTML(value="</br>")
html_logo = widgets.HTML(value=f"""
<div align="right">
    <img src='https://drive.google.com/uc?export=download&id=1PWyYWWESkAWbTBd8PR_t6-2KMxLsDS-B' width="10%" height ="" />
</div>
""")
    
    
# UI 컴포넌트 - tab_box1
html_cover_title = widgets.HTML(value=f"""
<div align="center">
  <h1>2022 KU 스마트캠퍼스 데이터톤</h1>
  <h1>분석보고서 및 개념증명서</h1>
</div>
""")
html_cover_content = widgets.HTML(value=f"""
<div align="center">
    <h3>9조 bada</h3>
    <h3>"고려대학교 학우와 안암동 주민을 위한, 그늘막쉼터의 입지 선정 제안"</h3>
    <p>경영학과 2017120029 김지욱</p>
    <p>경영학과 2017120066 김종현</p>
    <p>경영학과 2019120010 이지원</p>
    <p>경영학과 2010120134 조수현</p>
    </br>
    </br>
    </br>
    </br>
    <h3>2부. <교내를 위한 그늘막벤치의 최적설치장소 제안> 편</h3>
    </br>
    </br>
    </br>
    </br>
</div>
""")

#
tab_box1 = widgets.VBox(
    [
        html_br,
        html_cover_title,
        html_br,
        html_cover_content,
        html_br,
        html_logo
    ]
)



# UI 컴포넌트 - tab_box2
html_header = widgets.HTML(value=f"<h2>{tab_name[1]}</h2>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>본 단계에서는, <b>"교내를 위한 그늘막벤치의 최적설치장소"</b>가 표현된 <b>UI를 활용하기 위한 가이드라인</b>을 안내합니다.</li>
    <li>본 단계에서는, <b>"교내를 위한 그늘막벤치의 최적설치장소"</b>를 제안하기 위한 <b>가용데이터와 그 분석내용</b>을 안내합니다.</li>
  </ol>
</div>
""")

#
html_subtitle_1 = widgets.HTML(value=f"<h2>1. UI를 활용하기 위한 가이드라인</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <ol>
    <li>UI를 활용하기 위해서는, 강의인원 데이터를 입력하여 <b>강의건물별 강의인원 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 생성된 강의건물별 강의인원 인스턴스 객체를 활용한 시각화 분석<b>신규설치수량을 산출하는 메소드</b>를 호출해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 시각화에 필요한 <b>지도정보체계 인스턴스 객체</b>를 생성해야 합니다.</li>
    <li>UI를 활용하기 위해서는, 적절한 정보유형을 선택하여 <b>위치정보시각화 분석결과</b>를 출력할 수 있습니다.</li>
  </ol>
</div>
""")
#
html_subtitle_2 = widgets.HTML(value=f"<h2>2. 가용데이터와 그 분석내용</h2>")
html_subcontent_2 = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>단위 휴게공간당 교내인구 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>교내인구 대상 설문조사결과 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")

#
tab_box2 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        html_br,
        
        html_subtitle_2,
        html_subcontent_2,
        html_br,
        
        html_logo
    ]
)




# UI 컴포넌트 - tab_box2
html_header = widgets.HTML(value=f"<h1>{tab_name[1]}</h1>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>단위 휴게공간당 교내인구 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>교내인구 대상 설문조사결과 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")

html_subtitle_1 = widgets.HTML(value=f"<h2>1. 실내휴게공간 대비 교내활동인구</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <ol>
    <li>본 단계에서는, <b>"교내를 위한 그늘막벤치의 최적설치장소"</b>가 표현된 <b>UI를 활용하기 위한 가이드라인</b>을 안내합니다.</li>
  </ol>
</div>
""")

button_draw_map_5 = widgets.Button(description='지도 표시하기')
out_draw_map_5 = widgets.Output()
def on_button_draw_map_clicked_5(_):
    # global map_recommendation
    with out_draw_map_5:
        out_draw_map_5.clear_output()
        # print(f'기상청_생활기상지수 조회서비스(3.0) API - 열 지수 데이터')
        get_map_for_ratio_of_population_to_rest_area()
button_draw_map_5.on_click(on_button_draw_map_clicked_5)

#
tab_box3 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        button_draw_map_5,
        out_draw_map_5,
        
        
        html_logo
    ]
)




# UI 컴포넌트 - tab_box3
html_header = widgets.HTML(value=f"<h2>{tab_name[1]}</h2>")
html_content = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>단위 휴게공간당 교내인구 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>교내인구 대상 설문조사결과 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")

#
html_subtitle_1 = widgets.HTML(value=f"<h2>1. 단위 휴게공간당 교내인구 기준</h2>")
html_subcontent_1 = widgets.HTML(value="""
<div>
  <ol>
    <li>본 단계에서는, <b>"교내를 위한 그늘막벤치의 최적설치장소"</b>가 표현된 <b>UI를 활용하기 위한 가이드라인</b>을 안내합니다.</li>
    <li>본 단계에서는, <b>"교내를 위한 그늘막벤치의 최적설치장소"</b>를 제안하기 위한 <b>가용데이터와 그 분석내용</b>을 안내합니다.</li>
  </ol>
</div>
""")

#
toggle_option = ['교내인구 시계열-시각화 분석', '실내휴게공간 시각화 분석', '단위 휴게공간당 교내인구 시계열-시각화 분석']
toggle_button = widgets.ToggleButtons(options=toggle_option, tooltips=toggle_option, value=None)

#
def set_state(option):
    if option == '교내인구 시계열-시각화 분석':
        get_map_for_population()
    elif option == '실내휴게공간 시각화 분석':
        get_map_for_rest_area()
    elif option == '단위 휴게공간당 교내인구 시계열-시각화 분석':
        get_map_for_ratio_of_population_to_rest_area()
    else:
        return None
f_subcontent_1 = interactive(set_state, option=toggle_button);


#
html_subtitle_2 = widgets.HTML(value=f"<h2>2. 교내인구 대상 설문조사결과 기준</h2>")
html_subcontent_2 = widgets.HTML(value="""
<div>
  <ol>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>교내활동인구 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
    <li>가용데이터와 그 분석내용을 확인하기 위해서는, <b>설문조사결과 기준</b>와 관련된 <b>토글 옵션을 선택</b>하여 해당 분석내용을 확인할 수 있습니다.</li>
  </ol>
</div>
""")


#
toggle_option = ['4', '5', '6']
toggle_button = widgets.ToggleButtons(options=toggle_option, tooltips=toggle_option, value=None)

#
def set_state(option):
    if option == '4':
        get_map_for_population()
    elif option == '5':
        get_map_for_rest_area()
    elif option == '6':
        get_map_for_ratio_of_population_to_rest_area()
    else:
        return None
f_subcontent_2 = interactive(set_state, option=toggle_button);

#
tab_box4 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_subtitle_1,
        html_subcontent_1,
        f_subcontent_1,
        html_br,
        
        html_subtitle_2,
        html_subcontent_2,
        # f_subcontent_2,
        html_br,
        html_logo,
    ]
)


# # UI 컴포넌트 - tab_box5
# html_header = widgets.HTML(value=f"<h2>{tab_name[4][3:]}</h2>")
# html_cover_title = widgets.HTML(value=f"""
# <div align="center">
#   <h1>2022 KU 스마트캠퍼스 데이터톤</h1>
#   <h1>분석보고서 및 개념증명서</h1>
# </div>
# """)
# html_content = widgets.HTML(value=f"""
# <div align="center">
#     <h3>9조 bada</h3>
#     <h3>"고려대학교 학우와 안암동 주민을 위한, 그늘막쉼터의 입지 선정 제안"</h3>
#     <p>경영학과 2017120029 김지욱</p>
#     <p>경영학과 2017120066 김종현</p>
#     <p>경영학과 2019120010 이지원</p>
#     <p>경영학과 2010120134 조수현</p>
#     </br>
#     </br>
#     </br>
#     <h3>제1부. <교외를 위한 그늘막쉼터의 최적설치장소 제안> 편</h3>
#     </br>
#     </br>
# </div>
# """)

#
tab_box5 = widgets.VBox(
    [
        html_title,
        html_header,
        html_content,
        html_br,
        
        html_logo,
    ]
)


children = [tab_box1, tab_box2, tab_box3, tab_box4]
tab.children = children
display(tab)